# Import & config

In [ ]:
import requests
import os
import pandas as pd
from bs4 import BeautifulSoup
from utils.countries_infos import get_countries

headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36'}

: 

### 1. Get all countries

In [ ]:
countries = get_countries()[:5]

### 2. Extract data

In [ ]:
def extract_health_care(location):
  res = requests.get(location["health-care-url"], headers=headers)
  soup = BeautifulSoup(res.content, 'html.parser')

  table = soup.find("table", class_="table_builder_with_value_explanation data_wide_table")
  component_elements = table.select('td.columnWithName')
  percentage_elements = table.select('td.indexValueTd')
  
  locations = [location for component in component_elements]
  components = [component.text for component in component_elements]
  percentages = [percentage.text.split("\n")[0] for percentage in percentage_elements]

  return locations, components, percentages

### 3. Create csv file with data of countries

In [ ]:
def create_csv(countries):
  d = { "Country": [], "Component of health care surveyed": [], "Satisfaction": [] }

  for country in countries:
    print(f"Data for {country['name']} in loading ...")
    data = extract_health_care(country)

    country_names = data[0]
    components = data[1]
    percentages = data[2]

    d["Country"].extend(country_names)
    d["Component of health care surveyed"].extend(components)
    d["Satisfaction"].extend(percentages)


  df = pd.DataFrame(data=d)
  df.to_csv(os.path.join('bronze_data/', 'health.csv'), index=False)



In [ ]:
create_csv(countries)